### <Center> Лабораторна робота №9. <br> Оцінка інформативності ознак за допомогою градієнтного бустінгу

Детальних інструкцій не буде, завдання буде описано тезисно. Рекомендується використати метод Xgboost чи Catboost, та визначити інформативність ознак.

**Спочатку налаштуємо доступ до даних на google drive (якщо ви відкриваєте блокнот в google colab, а не на PC) шляхом монтування google drive**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive') 

Перевіримо шлях до папки з матеріалами лаборатоної роботи на google drive. Якщо у вас шлях відрізняється то відредагуйте

In [ ]:
!ls gdrive/'My Drive'/TEACHING/IntroDataScience/intro_to_data_science/Lab_7_8/

data
img
lab7_credit_scoring_random_forest.ipynb
lab7_credit_scoring_random_forest.pdf
lab8_quality_metrics_classification.ipynb
lab8_quality_metrics_classification.pdf
lab_9_xgb_flight_delays.ipynb


Перемістимо матеріали лабораторної роботи з google drive на віртуальну машину google colab

In [ ]:
!cp -a gdrive/'My Drive'/TEACHING/IntroDataScience/intro_to_data_science/Lab_7_8/. .
!ls

data
gdrive
img
lab7_credit_scoring_random_forest.ipynb
lab7_credit_scoring_random_forest.pdf
lab8_quality_metrics_classification.ipynb
lab8_quality_metrics_classification.pdf
lab_9_xgb_flight_delays.ipynb
sample_data


In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('data/flight_delays_train.csv')
test = pd.read_csv('data/flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Необхідно за часом вильоту літака, коду авіакомпанії-перевізника, місцю вильоту і прильоту і відстанню між аеропортами вильоту і прильоту зпрогнозувати затримку вильоту більше 15 хвилин. 

## Завдання 1. Створити вибірку :
* ознаки Distance і DepTime використати без змін;
* створити ознаку "маршрут" на  основі Origin та Dest;
* до ознак Month, DayofMonth, DayOfWeek, UniqueCarrier і "маршрут" застосувати OHE-перетворення (LabelBinarizer);

In [5]:
from sklearn.preprocessing import LabelBinarizer

def build_binary_features(data, column_name):
    binarizer = LabelBinarizer()
    values = binarizer.fit_transform(data[column_name])
    columns = (f"{column_name}_{unique_value}" for unique_value in binarizer.classes_)
    return pd.DataFrame(data=values, index=data.index, columns=columns)

In [6]:
train["Route"] = train["Origin"] + " -> " + train["Dest"]
features_to_binarize = ["Month", "DayofMonth", "DayOfWeek", "UniqueCarrier"]
binarized = [build_binary_features(train, feature) for feature in features_to_binarize]
X_train = pd.concat([train[["Distance", "DepTime"]]] + binarized, axis=1)
X_train.head()

,Distance,DepTime,Month_c-1,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,Month_c-5,...,UniqueCarrier_MQ,UniqueCarrier_NW,UniqueCarrier_OH,UniqueCarrier_OO,UniqueCarrier_TZ,UniqueCarrier_UA,UniqueCarrier_US,UniqueCarrier_WN,UniqueCarrier_XE,UniqueCarrier_YV
0,732,1934,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,834,1548,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,416,1422,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,872,1015,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,423,1828,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
from sklearn.feature_extraction import FeatureHasher

hasher = FeatureHasher(n_features=13, input_type="string")
values = hasher.fit_transform(train["Route"]).toarray()
route_columns = [f"route_{column_num}_feature" for column_num in range(13)]
X_train[route_columns] = values
X_train.head()

,Distance,DepTime,Month_c-1,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,Month_c-5,...,route_3_feature,route_4_feature,route_5_feature,route_6_feature,route_7_feature,route_8_feature,route_9_feature,route_10_feature,route_11_feature,route_12_feature
0,732,1934,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0
1,834,1548,0,0,0,0,0,0,1,0,...,1.0,0.0,0.0,0.0,0.0,1.0,-2.0,0.0,1.0,0.0
2,416,1422,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0
3,872,1015,0,0,1,0,0,0,0,0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,423,1828,0,1,0,0,0,0,0,0,...,2.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0


In [8]:
Y_train = train["dep_delayed_15min"].apply(lambda val: 1 if val == "Y" else 0).values
Y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## Завдання 2. Побудувати модель і оцінити інформативність ознак :
* побудувати модель на основі xgboost;
* налаштувати гіперпараметри з використанням крос-валідації;
* оцінити інформативність ознак.

In [9]:
from numpy.random import uniform
from sklearn.model_selection import GridSearchCV

model = XGBClassifier(seed=17)
params = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
best_model = GridSearchCV(model, params, n_jobs=-1, scoring="roc_auc", cv=5)
best_model.fit(X_train, Y_train)

D:\ANKND\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:40:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, seed=17,
                                     subsample=None, tree_met

In [10]:
best_features = pd.DataFrame(best_model.best_estimator_.feature_importances_, index=X_train.columns, columns=["Importance"]).sort_values(by="Importance", ascending=False)
best_features.head(20)

,Importance
DepTime,0.023179
DayofMonth_c-2,0.022202
UniqueCarrier_HA,0.021697
UniqueCarrier_EV,0.021583
UniqueCarrier_HP,0.020830
DayofMonth_c-3,0.019735
UniqueCarrier_FL,0.019332
UniqueCarrier_WN,0.017965
Month_c-7,0.017337
DayofMonth_c-6,0.015964
